In [2]:
from urllib.request import urlretrieve
from urllib.request import urlopen
from bs4 import BeautifulSoup

html = urlopen('http://www.pythonscraping.com')
bs = BeautifulSoup(html, 'html.parser')
imageLocation = bs.find('a',{'id':'logo'}).find('img')['src']
urlretrieve(imageLocation,'logo.jpg' )

# you know what nonetype means

AttributeError: 'NoneType' object has no attribute 'find'

In [ ]:
import os
from urllib.request import urlretrieve
from urllib.request import urlopen
from bs4 import BeautifulSoup

downloadDirectory = 'downloaded'
baseUrl = 'http://pythonscraping.com'

def getAbsoluteURL(baseUrl, source):
    if source.startswith('http://www.'):
        url = 'http://{}'.format(source[11:])
    elif source.startswith('http://'):
        url = source
    elif source.startswith('www.'):
        url = source[4:]
        url = 'http://{}'.format(source)
    else:
        url = '{}/{}'.format(baseUrl, source)
    if baseUrl not in url:
        return None
    return url

def getDownloadPath(baseUrl, absoluteUrl, downloadDirectory):
    path = absoluteUrl.replace('www.', '')
    path = path.replace(baseUrl, '')
    path = downloadDirectory+path
    directory = os.path.dirname(path)

    if not os.path.exists(directory):
        os.makedirs(directory)

    return path

html = urlopen('http://www.pythonscraping.com')
bs = BeautifulSoup(html, 'html.parser')
downloadList = bs.findAll(src=True)

for download in downloadList:
    fileUrl = getAbsoluteURL(baseUrl, download['src'])
    if fileUrl is not None:
        print(fileUrl)

urlretrieve(fileUrl, getDownloadPath(baseUrl, fileUrl, downloadDirectory))

# don't run this code it may download malicious files 

In [ ]:
import csv

csvFile = open('test.csv', 'w+')
try:
    writer = csv.writer(csvFile)
    writer.writerow(('number', 'number plus 2', 'number times 2'))
    for i in range(10):
        writer.writerow( (i, i+2, i*2))
finally:
    csvFile.close()

In [ ]:
import csv
from urllib.request import urlopen
from bs4 import BeautifulSoup

html = urlopen('http://en.wikipedia.org/wiki/Comparison_of_text_editors')
bs = BeautifulSoup(html, 'html.parser')
# The main comparison table is currently the first table on the page
table = bs.findAll('table',{'class':'wikitable'})[0]
rows = table.findAll('tr')

csvFile = open('editors.csv', 'wt+')
writer = csv.writer(csvFile)
try:
    for row in rows:
        csvRow = []
        for cell in row.findAll(['td', 'th']):
            csvRow.append(cell.get_text())
        writer.writerow(csvRow)
finally:
    csvFile.close()

In [5]:
import csv
from urllib.request import urlopen
from bs4 import BeautifulSoup

html = urlopen('http://en.wikipedia.org/wiki/Comparison_of_text_editors')
bs = BeautifulSoup(html, 'html.parser')

# The main comparison table is currently the first table on the page

table = bs.findAll('table',{'class':'wikitable'})[0]
rows = table.findAll('tr')

csvFile = open('editor.csv', 'wt+')
writer = csv.writer(csvFile)
try:
    for row in rows:
        csvRow = []
        for cell in row.findAll(['td', 'th']):
            csvRow.append(cell.get_text())
        writer.writerow(csvRow)
finally:
    csvFile.close()
        

In [ ]:
import requests
import pandas as pd

res = requests.get('http://en.wikipedia.org/wiki/Comparison_of_text_editors')
dfs = pd.read_html(res.text)
df = dfs[0]
df.to_csv('editor.csv',index=False)

In [14]:
import mysql.connector
import pandas as pd

# using the mysql.connector is a good alternative to pymysql
mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  password="karuga12"
)

mycursor = mydb.cursor()

mycursor.execute("SHOW DATABASES")

for x in mycursor:
  print(x)

# close the connection 

mycursor.close()
mydb.close()

('banking_system',)
('Clinic',)
('employee',)
('fashion',)
('freedom_insurance',)
('ICS2304',)
('information_schema',)
('mysql',)
('performance_schema',)
('pets',)
('sys',)
('university',)


In [18]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import datetime
import  random 
import  pymysql
import  re

conn = pymysql.connect(
    host='127.0.0.1',
    unix_socket='/tmp/mysql.sock',
    user='root',
    passwd='karuga12',
    db='mysql',
    charset='utf8'
)

cur = conn.cursor()
# create the database first 
cur.execute('CREATE DATABASE IF NOT EXISTS scraping;')
# use the database 
cur.execute('USE Scraping;')

def store(title, content):
    # create the table to insert to
    cur.execute(
        """CREATE TABLE IF NOT EXISTS pages(
            id BIGINT(7) NOT NULL AUTO_INCREMENT,
            title VARCHAR(255),
            content VARCHAR(10000),
            created TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
            PRIMARY KEY(id));
            """)

    # insert into the pages table
    cur.execute('INSERT INTO pages (title, content) VALUES ("%s", "%s")', (title, content))
    #  commiting the changes
    cur.connection.commit()

def getLinks(articleUrl):
    html = urlopen('http://en.wikipedia.org'+articleUrl)
    bs = BeautifulSoup(html, 'html.parser')
    title = bs.find('h1').get_text()
    content = bs.find('div', {'id':'mw-content-text'}).find('p').get_text()
    store(title, content)
    return bs.find('div', {'id':'bodyContent'}).findAll('a', href=re.compile('^(/wiki/)((?!:).)*$'))

links = getLinks('/wiki/Kevin_Bacon')
try:
    while len(links) > 0:
         newArticle = links[random.randint(0, len(links)-1)].attrs['href']
         print(newArticle)
         links = getLinks(newArticle)
finally:
    cur.close()
    conn.close()


/wiki/Stir_of_Echoes
/wiki/Judy_Hofflund
/wiki/The_Matchmaker_(1997_film)
/wiki/On_Raglan_Road
/wiki/The_Dubliners_discography
/wiki/Compact_Disc
/wiki/MIDI
/wiki/Coherent_Accelerator_Processor_Interface
/wiki/PCI-X
/wiki/ProLiant
/wiki/HP_Data_Protector
/wiki/Compaq_Presario
/wiki/HPE_Integrity_Servers
/wiki/HP_TouchSmart
/wiki/Indigo_Digital_Press
/wiki/OpenVMS
/wiki/List_of_operating_systems
/wiki/Pascal_(programming_language)
/wiki/ISO_639-2
/wiki/XML_Metadata_Interchange
/wiki/MP4_file_format
/wiki/ISO_16750
/wiki/Office_Open_XML
/wiki/ISO/IEC_8859-16
/wiki/Mac_OS_Celtic
/wiki/Code_page_1014
/wiki/LST_1564
/wiki/Mac_OS_Cyrillic_encoding
/wiki/National_Replacement_Character_Set
/wiki/Code_page_858
/wiki/ISO_5428
/wiki/ISO_15189
/wiki/Meta-Object_Facility
/wiki/Shoe_size
/wiki/ISO_10303-28
/wiki/ISO_9000
/wiki/TIFF/EP
/wiki/FLIC_(file_format)
/wiki/File_format
/wiki/Thumbnail
/wiki/Noun
/wiki/ISBN_(identifier)
/wiki/International_Article_Number#Check_digit
/wiki/ISMN
/wiki/Prontor-C

KeyboardInterrupt: 

In [67]:
# create the  database and the tables
import mysql.connector
import pandas as pd

# using the mysql.connector is a good alternative to pymysql
mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  password="karuga12"
)

mycursor = mydb.cursor()

mycursor.execute("SHOW DATABASES")

for x in mycursor:
  print(x)

# close the connection 
mycursor.close()
mydb.close()

('banking_system',)
('Clinic',)
('employee',)
('fashion',)
('freedom_insurance',)
('ICS2304',)
('information_schema',)
('mysql',)
('performance_schema',)
('pets',)
('scraping',)
('sys',)
('university',)
('wikipedia',)


In [69]:
def check_db_update(Database,table,time=20):
    import mysql.connector
    import pandas as pd
    #import time

    # creating the database connection
    mydb = mysql.connector.connect(
        host="localhost",
        user="root",
        password=input('Enter your password'),
        database=Database
    )

    # reading content from the selected table using pandas 
    table_data = pd.read_sql(f'SELECT * FROM {table}',mydb)
    #table_data.head(10)

    if time >= time:
        # close the connections
        mycursor.close()
        mydb.close()


    return table_data.head(20)


In [70]:
check_db_update(Database='scraping',table='pages')

,id,title,content,created
0,1,'Kevin Bacon','\n',2023-01-18 23:48:53
1,2,'Stir of Echoes','Stir of Echoes is a 1999 American supernatura...,2023-01-18 23:48:57
2,3,'Judy Hofflund','Judy Hofflund is a US film producer. She work...,2023-01-18 23:49:03
3,4,'The Matchmaker (1997 film)','\n',2023-01-18 23:49:05
4,5,'On Raglan Road','\n',2023-01-18 23:49:17
5,6,'The Dubliners discography','\n',2023-01-18 23:49:21
6,7,'Compact disc','\n',2023-01-18 23:49:27
7,8,'MIDI','\n',2023-01-18 23:49:30
8,9,'Coherent Accelerator Processor Interface','Coherent Accelerator Processor Interface (CAP...,2023-01-18 23:49:32
9,10,'PCI-X',"'PCI-X, short for Peripheral Component Interco...",2023-01-18 23:49:35


In [35]:
# create the  database and the tables
import mysql.connector
import pandas as pd

# using the mysql.connector is a good alternative to pymysql
mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  password="karuga12"
)

mycursor = mydb.cursor()

In [41]:
# creating the database 
#mycursor.execute('CREATE DATABASE IF NOT EXISTS wikipedia;')

# selecting database to use 
mycursor.execute('USE wikipedia;')

# create the pages table
mycursor.execute("""
CREATE TABLE IF NOT EXISTS `wikipedia`.`pages` (
`id` INT NOT NULL AUTO_INCREMENT,
`url` VARCHAR(255) NOT NULL,
`created` TIMESTAMP NOT NULL DEFAULT CURRENT_TIMESTAMP, 
PRIMARY KEY (`id`));
)
"""
)

# create the links table
mycursor.execute("""
CREATE TABLE IF NOT EXISTS `wikipedia`.`links` ( 
`id` INT NOT NULL AUTO_INCREMENT, 
`fromPageId` INT NULL, 
`toPageId` INT NULL,
`created` TIMESTAMP NOT NULL DEFAULT CURRENT_TIMESTAMP, 
PRIMARY KEY (`id`));
)
""")

In [42]:
# close the connection 
mycursor.close()
mydb.close()

In [43]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re
import pymysql
from random import shuffle

conn = pymysql.connect(
    host='127.0.0.1', 
    unix_socket='/tmp/mysql.sock',
    user='root', 
    passwd='karuga12', 
    db='mysql', 
    charset='utf8')

cur = conn.cursor()
cur.execute('USE wikipedia')

def insertPageIfNotExists(url):
    cur.execute('SELECT * FROM pages WHERE url = %s', (url))
    if cur.rowcount == 0:
        cur.execute('INSERT INTO pages (url) VALUES (%s)', (url))
        conn.commit()
        return cur.lastrowid
    else:
        return cur.fetchone()[0]

def loadPages():
    cur.execute('SELECT * FROM pages')
    pages = [row[1] for row in cur.fetchall()]
    return pages

def insertLink(fromPageId, toPageId):
    cur.execute('SELECT * FROM links WHERE fromPageId = %s AND toPageId = %s', 
                  (int(fromPageId), int(toPageId)))
    if cur.rowcount == 0:
        cur.execute('INSERT INTO links (fromPageId, toPageId) VALUES (%s, %s)', 
                    (int(fromPageId), int(toPageId)))
        conn.commit()
def pageHasLinks(pageId):
    cur.execute('SELECT * FROM links WHERE fromPageId = %s', (int(pageId)))
    rowcount = cur.rowcount
    if rowcount == 0:
        return False
    return True

def getLinks(pageUrl, recursionLevel, pages):
    if recursionLevel > 4:
        return

    pageId = insertPageIfNotExists(pageUrl)
    html = urlopen('http://en.wikipedia.org{}'.format(pageUrl))
    bs = BeautifulSoup(html, 'html.parser')
    links = bs.findAll('a', href=re.compile('^(/wiki/)((?!:).)*$'))
    links = [link.attrs['href'] for link in links]

    for link in links:
        linkId = insertPageIfNotExists(link)
        insertLink(pageId, linkId)
        if not pageHasLinks(linkId):
            print("PAGE HAS NO LINKS: {}".format(link))
            pages.append(link)
            getLinks(link, recursionLevel+1, pages)
        
        
getLinks('/wiki/Kevin_Bacon', 0, loadPages()) 
cur.close()
conn.close()

PAGE HAS NO LINKS: /wiki/Kevin_Bacon_(disambiguation)
PAGE HAS NO LINKS: /wiki/Main_Page
PAGE HAS NO LINKS: /wiki/Wikipedia
PAGE HAS NO LINKS: /wiki/Wikipedia_(disambiguation)
PAGE HAS NO LINKS: /wiki/274301_Wikipedia
PAGE HAS NO LINKS: /wiki/Wikipedia_community
PAGE HAS NO LINKS: /wiki/Jireel
PAGE HAS NO LINKS: /wiki/Jean_Deaux
PAGE HAS NO LINKS: /wiki/Wikipedia_logo
PAGE HAS NO LINKS: /wiki/Logo
PAGE HAS NO LINKS: /wiki/Online_encyclopedia
PAGE HAS NO LINKS: /wiki/Globe
PAGE HAS NO LINKS: /wiki/Jigsaw_puzzle
PAGE HAS NO LINKS: /wiki/Glyph
PAGE HAS NO LINKS: /wiki/Writing_system
PAGE HAS NO LINKS: /wiki/Open_source
PAGE HAS NO LINKS: /wiki/Linux_Libertine
PAGE HAS NO LINKS: /wiki/Flag_of_the_United_States
PAGE HAS NO LINKS: /wiki/Flag_of_the_United_States
PAGE HAS NO LINKS: /wiki/Jimmy_Wales
PAGE HAS NO LINKS: /wiki/Nupedia
PAGE HAS NO LINKS: /wiki/Euclid_and_his_Modern_Rivals
PAGE HAS NO LINKS: /wiki/Lewis_Carroll
PAGE HAS NO LINKS: /wiki/Fisheye_lens
PAGE HAS NO LINKS: /wiki/Thomas_

KeyboardInterrupt: 

In [71]:
check_db_update(Database='wikipedia',table='pages')

,id,url,created
0,1,/wiki/Kevin_Bacon,2023-01-19 00:32:21
1,2,/wiki/Kevin_Bacon_(disambiguation),2023-01-19 00:32:27
2,3,/wiki/Main_Page,2023-01-19 00:32:29
3,4,/wiki/Wikipedia,2023-01-19 00:32:31
4,5,/wiki/Wikipedia_(disambiguation),2023-01-19 00:32:40
5,6,/wiki/274301_Wikipedia,2023-01-19 00:32:41
6,7,/wiki/Wikipedia_community,2023-01-19 00:32:41
7,8,/wiki/Jireel,2023-01-19 00:32:41
8,9,/wiki/Jean_Deaux,2023-01-19 00:32:41
9,10,/wiki/Wikipedia_logo,2023-01-19 00:32:41


In [72]:
check_db_update(Database='wikipedia',table='links')

,id,fromPageId,toPageId,created
0,1,1,2,2023-01-19 00:32:27
1,2,2,3,2023-01-19 00:32:29
2,3,3,3,2023-01-19 00:32:31
3,4,3,4,2023-01-19 00:32:31
4,5,4,3,2023-01-19 00:32:40
5,6,4,4,2023-01-19 00:32:40
6,7,4,5,2023-01-19 00:32:40
7,8,5,4,2023-01-19 00:32:41
8,9,5,6,2023-01-19 00:32:41
9,10,5,7,2023-01-19 00:32:41


In [ ]:
import smtplib
from email.mime.text import MIMEText

msg = MIMEText('The body of the email is here')

msg['Subject'] = 'An Email Alert'
msg['From'] = 'ryan@pythonscraping.com'
msg['To'] = 'webmaster@pythonscraping.com'

s = smtplib.SMTP('localhost')
s.send_message(msg)
s.quit()

In [ ]:
import smtplib 
from email.mime.text import MIMEText

msg = MIMEText('Body: You have done very well today keep going champ.')

msg['Subject'] = 'An Email Alert'
msg['From'] = 'oscarkaruga066@gmail.com'
msg['To'] =  'oscarkaruga1@gmail.com'

s = smtplib.SMTP('localhost')
s.send_message(msg)
s.quit()

In [48]:
import smtplib
from email.mime.text import MIMEText
from bs4 import BeautifulSoup
from urllib.request import urlopen
import time

def sendMail(subject, body):
    msg = MIMEText(body)
    msg['Subject'] = subject
    msg['From'] ='christmas_alerts@pythonscraping.com'
    msg['To'] = 'oscarkaruga1@gmail.com'

    s = smtplib.SMTP('localhost')
    s.send_message(msg)
    s.quit()

bs = BeautifulSoup(urlopen('https://isitchristmas.com/'), 'html.parser')
while(bs.find('a', {'id':'answer'}).attrs['title'] == 'NO'):
    print('It is not Christmas yet.')
    time.sleep(3600)
    bs = BeautifulSoup(urlopen('https://isitchristmas.com/'), 'html.parser')
sendMail('It\'s Christmas!', 
         'According to http://itischristmas.com, it is Christmas!')

ConnectionRefusedError: [Errno 61] Connection refused

You need to set a smtp server for  sending the emails

## Reading Documents

In [73]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
textPage = urlopen('http://www.pythonscraping.com/pages/warandpeace/chapter1.txt')
print(textPage.read())

b'CHAPTER I\n\n"Well, Prince, so Genoa and Lucca are now just family estates of theBuonapartes. But I warn you, if you don\'t tell me that this means war,if you still try to defend the infamies and horrors perpetrated bythat Antichrist- I really believe he is Antichrist- I will havenothing more to do with you and you are no longer my friend, no longermy \'faithful slave,\' as you call yourself! But how do you do? I seeI have frightened you- sit down and tell me all the news."\n\nIt was in July, 1805, and the speaker was the well-known AnnaPavlovna Scherer, maid of honor and favorite of the Empress MaryaFedorovna. With these words she greeted Prince Vasili Kuragin, a manof high rank and importance, who was the first to arrive at herreception. Anna Pavlovna had had a cough for some days. She was, asshe said, suffering from la grippe; grippe being then a new word inSt. Petersburg, used only by the elite.\n\nAll her invitations without exception, written in French, anddelivered by a scarle

In [ ]:
from urllib.request import urlopen
textPage = urlopen(
             'http://www.pythonscraping.com/pages/warandpeace/chapter1-ru.txt')
print(textPage.read())

In [75]:
from urllib.request import urlopen

textPage = urlopen(
             'http://www.pythonscraping.com/pages/warandpeace/chapter1-ru.txt')
print(str(textPage.read(), 'utf-8'))

ЧАСТЬ ПЕРВАЯ

I

— Eh bien, mon prince. Gênes et Lucques ne sont plus que des apanages, des поместья, de la famille Buonaparte. Non, je vous préviens que si vous ne me dites pas que nous avons la guerre, si vous vous permettez encore de pallier toutes les infamies, toutes les atrocités de cet Antichrist (ma parole, j'y crois) — je ne vous connais plus, vous n'êtes plus mon ami, vous n'êtes plus мой верный раб, comme vous dites. Ну, здравствуйте, здравствуйте. Je vois que je vous fais peur, садитесь и рассказывайте.
Так говорила в июле 1805 года известная Анна Павловна Шерер, фрейлина и приближенная императрицы Марии Феодоровны, встречая важного и чиновного князя Василия, первого приехавшего на ее вечер. Анна Павловна кашляла несколько дней, у нее был грипп, как она говорила (грипп был тогда новое слово, употреблявшееся только редкими). В записочках, разосланных утром с красным лакеем, было написано без различия во всех:
«Si vous n'avez rien de mieux à faire, Monsieur le comte (или mon 

In [76]:
html = urlopen("http://en.wikipedia.org/wiki/Python_(programming_language)")
bs = BeautifulSoup(html, "html.parser")
content = bs.find("div", {"id":"mw-content-text"}).get_text()
content = bytes(content, "UTF-8")
content = content.decode("UTF-8")
print(content)



General-purpose programming language


PythonParadigmMulti-paradigm: object-oriented,[1] procedural (imperative), functional, structured, reflectiveDesigned byGuido van RossumDeveloperPython Software FoundationFirst appeared20 February 1991; 31 years ago (1991-02-20)[2]Stable release3.11.1[3] 
   / 6 December 2022; 43 days ago (6 December 2022)Preview release3.12.0a4[4] 
   / 10 January 2023; 8 days ago (10 January 2023)
Typing disciplineDuck, dynamic, strong typing;[5] gradual (since 3.5, but ignored in CPython)[6]OSWindows, macOS, Linux/UNIX, Android[7][8] and more[9]LicensePython Software Foundation LicenseFilename extensions.py, .pyi, .pyc, .pyd, .pyw, .pyz (since 3.5),[10] .pyo (prior to 3.5)[11]Websitepython.orgMajor implementationsCPython, PyPy, Stackless Python, MicroPython, CircuitPython, IronPython, JythonDialectsCython, RPython, Starlark[12]Influenced byABC,[13] Ada,[14] ALGOL 68,[15] APL,[16] C,[17] C++,[18] CLU,[19] Dylan,[20] Haskell,[21][16] Icon,[22] Lisp,[23] Modula-

In [77]:
from urllib.request import urlopen
from io import StringIO
import csv

data = urlopen('http://pythonscraping.com/files/MontyPythonAlbums.csv').read().decode('ascii', 'ignore')
dataFile = StringIO(data)
csvReader = csv.reader(dataFile)

for row in csvReader:
    print(row)
    print("The album \""+row[0]+"\" was released in "+str(row[1]))

['Name', 'Year']
The album "Name" was released in Year
["Monty Python's Flying Circus", '1970']
The album "Monty Python's Flying Circus" was released in 1970
['Another Monty Python Record', '1971']
The album "Another Monty Python Record" was released in 1971
["Monty Python's Previous Record", '1972']
The album "Monty Python's Previous Record" was released in 1972
['The Monty Python Matching Tie and Handkerchief', '1973']
The album "The Monty Python Matching Tie and Handkerchief" was released in 1973
['Monty Python Live at Drury Lane', '1974']
The album "Monty Python Live at Drury Lane" was released in 1974
['An Album of the Soundtrack of the Trailer of the Film of Monty Python and the Holy Grail', '1975']
The album "An Album of the Soundtrack of the Trailer of the Film of Monty Python and the Holy Grail" was released in 1975
['Monty Python Live at City Center', '1977']
The album "Monty Python Live at City Center" was released in 1977
['The Monty Python Instant Record Collection', '1977

In [ ]:
for row in csvReader:
    print("The album \""+row[0]+"\" was released in "+str(row[1]))
from urllib.request import urlopen
from io import StringIO
import csv

data = urlopen("http://pythonscraping.com/files/MontyPythonAlbums.csv").read().decode('ascii', 'ignore')
dataFile = StringIO(data)
dictReader = csv.DictReader(dataFile)

print(dictReader.fieldnames)

for row in dictReader:
    print(row)

using csv.DictReader takes longer 

In [79]:
! pip install pdfminer3k

     |████████████████████████████████| 100 kB 210 kB/s ta 0:00:01


In [80]:
from urllib.request import urlopen
from pdfminer.pdfinterp import PDFResourceManager, process_pdf
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from io import StringIO
from io import open

def readPDF(pdfFile):
    rsrcmgr = PDFResourceManager()
    retstr = StringIO()
    laparams = LAParams()
    device = TextConverter(rsrcmgr, retstr, laparams=laparams)

    process_pdf(rsrcmgr, device, pdfFile)
    device.close()

    content = retstr.getvalue()
    retstr.close()
    return content

pdfFile = urlopen("http://pythonscraping.com/pages/warandpeace/chapter1.pdf")
outputString = readPDF(pdfFile)
print(outputString)
pdfFile.close()

CHAPTER I

"Well, Prince, so Genoa and Lucca are now just family estates of 
theBuonapartes. But I warn you, if you don't tell me that this 
means war,if you still try to defend the infamies and horrors 
perpetrated bythat Antichrist- I really believe he is Antichrist- I will 
havenothing more to do with you and you are no longer my friend, 
no longermy 'faithful slave,' as you call yourself! But how do you 
do? I seeI have frightened you- sit down and tell me all the news."

It was in July, 1805, and the speaker was the well-known 
AnnaPavlovna Scherer, maid of honor and favorite of the 
Empress MaryaFedorovna. With these words she greeted Prince 
Vasili Kuragin, a manof high rank and importance, who was the 
first to arrive at herreception. Anna Pavlovna had had a cough for 
some days. She was, asshe said, suffering from la grippe; grippe 
being then a new word inSt. Petersburg, used only by the elite.

All her invitations without exception, written in French, 
anddelivered by a scar

In [81]:
from zipfile import ZipFile
from urllib.request import urlopen
from io import BytesIO
from bs4 import BeautifulSoup

wordFile = urlopen('http://pythonscraping.com/pages/AWordDocument.docx').read()
wordFile = BytesIO(wordFile)
document = ZipFile(wordFile)
xml_content = document.read('word/document.xml')

wordObj = BeautifulSoup(xml_content.decode('utf-8'), 'xml')
textStrings = wordObj.find_all('w:t')

for textElem in textStrings:
    print(textElem.text)


A Word Document on a Website
This is a Word document, full of content that you want very much. Unfortunately, it’s difficult to access because I’m putting it on my website as a .
docx
 file, rather than just publishing it as HTML


In [82]:
textStrings = wordObj.find_all('w:t')

for textElem in textStrings:
    style = textElem.parent.parent.find('w:pStyle')
    if style is not None and style['w:val'] == 'Title':
        print('Title is: {}'.format(textElem.text))
    else:
        print(textElem.text)

Title is: A Word Document on a Website
This is a Word document, full of content that you want very much. Unfortunately, it’s difficult to access because I’m putting it on my website as a .
docx
 file, rather than just publishing it as HTML


## Cleaning Dirty data 